# check new slicing information

In [ ]:
%reset

In [1]:
import copy  # copy big/deep objects by value
import csv
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import math
import os  # OS operations (read/write files/folders)
import sys
import time
import warnings  # hide warnings
from collections import Counter
from itertools import groupby
import matplotlib

# process parallelization
from multiprocessing import Manager, Pool, RawArray, cpu_count
from os.path import exists

import matplotlib.pyplot as plt  # mother of plots focr Python

# import mlxtend
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes


import scipy.signal

In [2]:
# datapath = "E:/Cyprus_project_overview/data/ETdata/oldData/final_labeling_data_frame_nr_NOT_sorted"
datapath = "E:/Cyprus_project_overview/data/ETdata/labeled_ET_data_sorted"
# savepathGraphPrep = "E:/Cyprus_project_overview/data/graphs/graphPrepETdata"

In [3]:
# slicing info
slicingpath = "E:/Cyprus_project_overview/data"

slicingList = pd.read_csv(os.path.join(slicingpath, "fixation_adj_start_end_ns.csv"))
slicingList.head()

,session,adjusted_start_frame,adjusted_start_timestamp,adjusted_start_timestamp_ns,adjusted_end_frame,adjusted_end_timestamp,adjusted_end_timestamp_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,13670,2023-09-05 09:03:52.544755555,1693904632544755555,32924,2023-09-05 09:14:34.520766666,1693905274520766666
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,9867,2023-09-05 09:39:53.840633333,1693906793840633333,28818,2023-09-05 09:50:25.712655555,1693907425712655555
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,8046,2023-09-05 10:14:30.260911111,1693908870260911111,29536,2023-09-05 10:26:26.792900000,1693909586792900000
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,11600,2023-09-06 07:43:04.403811111,1693986184403811111,30692,2023-09-06 07:53:40.979755555,1693986820979755555
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,8620,2023-09-06 08:13:24.851122222,1693988004851122222,27977,2023-09-06 08:24:10.263077777,1693988650263077777


In [82]:
# slicing info Paula
slicingpathP = "E:/Cyprus_project_overview/store_cyprus_project/pvondrlik/data"

slicingListP = pd.read_csv(os.path.join(slicingpathP, "Cyprus_start_end_frames_time.csv"))
slicingListP.head()

,session,start_frame,end_frame,start_time,end_time
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,13670,32920,1693904632544755555,1693905274388766666
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,9880,28805,1693906794272600000,1693907425280677777
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,8060,29530,1693908870728877777,1693909586593522222
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,11600,30680,1693986184403811111,1693986820579733333
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,8620,27960,1693988004851122222,1693988649695222222


In [85]:
slicingListP["sanityCheckTS"] = slicingListP["start_time"] - slicingListP["end_time"]

# sanity check the start time slicing timestamps - in general, the old/P ts must be smaller than the new one (since the new one must start later)
# or equally ts (old and new)
slicingListP["sanityCheck_startTime"] = slicingListP["start_time"] <= slicingList["adjusted_start_timestamp_ns"]
slicingListP["difference_startTime (s)"] = (slicingListP["start_time"] - slicingList["adjusted_start_timestamp_ns"])/1e9


# sanity check the end time slicing timestamps - in general, the new ts must be smaller than the old/P one (since the old one must end later)
# or equally ts (old and new)
slicingListP["sanityCheck_endTime"] =  slicingList["adjusted_end_timestamp_ns"] <= slicingListP["end_time"]
slicingListP["difference_endTime (s)"] =  (slicingList["adjusted_end_timestamp_ns"] - slicingListP["end_time"])/1e9


print(11-13)
slicingListP.head(13)

-2


,session,start_frame,end_frame,start_time,end_time,sanityCheckTS,sanityCheck_startTime,difference_startTime (s),sanityCheck_endTime,difference_endTime (s)
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,13670,32920,1693904632544755555,1693905274388766666,-641844011111,True,0.000000,False,0.132000
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,9880,28805,1693906794272600000,1693907425280677777,-631008077777,False,0.431967,False,0.431978
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,8060,29530,1693908870728877777,1693909586593522222,-715864644445,False,0.467967,False,0.199378
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,11600,30680,1693986184403811111,1693986820579733333,-636175922222,True,0.000000,False,0.400022
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,8620,27960,1693988004851122222,1693988649695222222,-644844100000,True,0.000000,False,0.567856
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,12160,31450,1693989967720244444,1693990610896299999,-643176055555,True,0.000000,False,0.164100
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,8890,28170,1693996180867344444,1693996823707288888,-642839944444,False,1.067889,False,0.368033
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,8710,27720,1693998168175677777,1693998802015900000,-633840222223,False,1.800433,False,0.535244
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,7640,26950,1693999974960144444,1694000618804288888,-643844144444,True,0.000000,False,0.299900
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,8600,27820,1694014580757844444,1694015221735855555,-640978011111,False,0.063978,False,0.132133


In [4]:
datapathF = "E:/Cyprus_project_overview/my_data_backup/eye tracking data_backup"
data_foldersF = sorted([f for f in os.listdir(datapathF) if not f.startswith('TestS')], key=str.lower)
print(data_foldersF)


['Expl_1_ET_1_2023-09-05_11-56-16_ET', 'Expl_1_ET_2_2023-09-05_12-34-24_ET', 'Expl_1_ET_3_2023-09-05_13-10-01_ET', 'Expl_2_ET_1_2023-09-06_10-36-37_ET', 'Expl_2_ET_2_2023-09-06_11-08-36_ET', 'Expl_2_ET_3_2023-09-06_11-39-21_ET', 'Expl_3_ET_1_2023-09-06_13-24-43_ET', 'Expl_3_ET_2_2023-09-06_13-57-57_ET', 'Expl_3_ET_3_2023-09-06_14-28-39_ET', 'Expl_4_ET_1_2023-09-06_18-31-33_ET', 'Expl_4_ET_2_2023-09-06_18-57-24_ET', 'Expl_5_ET_1_2023-09-07_18-17-19_ET', 'Expl_5_ET_2_2023-09-07_18-48-26_ET']


In [9]:
labledDataPath = "E:/Cyprus_project_overview/store_cyprus_project/pvondrlik/data/results_cyprus"
allLabledData = pd.read_csv(os.path.join(labledDataPath, "fixation_and_labels_total.csv"))
allLabledData.head()

,section_id,recording_id,timestamp_[ns],frame_nr,session,x,y,start_timestamp_ns,end_timestamp_ns,duration_ms,fixation_x,fixation_y,azimuth_[deg],fixation_id,fixation_sum_label,level_sum_annotation,fixation_pXc_label,level_pXc_annotation,session_duration
0,e1b51f12-ca55-46ad-b9ab-367ad8a4f825,72b5f845-3c76-473b-b8b0-5bec13a5cb4d,1693904632744766666,13676,Expl_1_ET_1_2023-09-05_11-56-16_ET,1161.0,874.0,1693904632719606803,1693904632794434803,74,1188.792,861.039,24.166327,964,street,1.0,street,1.0,6.416136e+11
1,e1b51f12-ca55-46ad-b9ab-367ad8a4f825,72b5f845-3c76-473b-b8b0-5bec13a5cb4d,1693904632744766666,13676,Expl_1_ET_1_2023-09-05_11-56-16_ET,1191.0,867.0,1693904632719606803,1693904632794434803,74,1188.792,861.039,24.166327,964,street,1.0,street,1.0,6.416136e+11
2,e1b51f12-ca55-46ad-b9ab-367ad8a4f825,72b5f845-3c76-473b-b8b0-5bec13a5cb4d,1693904632744766666,13676,Expl_1_ET_1_2023-09-05_11-56-16_ET,1204.0,856.0,1693904632719606803,1693904632794434803,74,1188.792,861.039,24.166327,964,street,1.0,street,1.0,6.416136e+11
3,e1b51f12-ca55-46ad-b9ab-367ad8a4f825,72b5f845-3c76-473b-b8b0-5bec13a5cb4d,1693904632744766666,13676,Expl_1_ET_1_2023-09-05_11-56-16_ET,1173.0,872.0,1693904632719606803,1693904632794434803,74,1188.792,861.039,24.166327,964,street,1.0,street,1.0,6.416136e+11
4,e1b51f12-ca55-46ad-b9ab-367ad8a4f825,72b5f845-3c76-473b-b8b0-5bec13a5cb4d,1693904632744766666,13676,Expl_1_ET_1_2023-09-05_11-56-16_ET,1280.0,821.0,1693904632719606803,1693904632794434803,74,1188.792,861.039,24.166327,964,street,1.0,street,1.0,6.416136e+11


In [65]:
def analyse_start_slice(
    session_name: str,
    slice_start_ts: int,
    slice_end_ts: int
):

    # ------------------------------------------------------------------
    # 1️⃣  Isolate the required session
    # ------------------------------------------------------------------
    label_col = "fixation_sum_label"
    
    sess_mask = allLabledData["session"].astype(str).str.startswith(session_name)
    sess_df = allLabledData.loc[sess_mask].copy()
    
    if sess_df.empty:
        raise ValueError(f"No rows found for session '{session_name}'")


    # ------------------------------------------------------------------
    # 2️⃣  First row of the session – this is the reference point
    # ------------------------------------------------------------------
    session_start_ts = sess_df.iloc[0]["timestamp_[ns]"]

    # ------------------------------------------------------------------
    # 3️⃣  Does the slice start *before* the first data point?
    # ------------------------------------------------------------------
    slice_before_data_start = slice_start_ts < session_start_ts

    # ------------------------------------------------------------------
    # 4️⃣  Define the part of the slice we really can look at
    # ------------------------------------------------------------------
    # If the slice begins before the first fixation we can only start at the
    # first fixation timestamp.
    effective_start_ts = max(slice_start_ts, session_start_ts)

    # ------------------------------------------------------------------
    # 5️⃣  Select rows that lie inside the (possibly trimmed) slice
    # ------------------------------------------------------------------
    mask = (
        (sess_df["timestamp_[ns]"] >= effective_start_ts) &
        (sess_df["timestamp_[ns]"] < slice_end_ts)
    )
    rows_in_slice = sess_df.loc[mask]

    # ------------------------------------------------------------------
    # 6️⃣  Unique labels inside the slice
    # ------------------------------------------------------------------
    unique_labels = rows_in_slice[label_col].dropna().unique().tolist()
    unique_label_count = len(unique_labels)   
    unique_fixationIDs = rows_in_slice["fixation_id"].dropna().unique().tolist()


    # ------------------------------------------------------------------
    # 7️⃣  Build the result dictionary
    # ------------------------------------------------------------------
    result = {
        "session": session_name,
        "slice_start_ns": slice_start_ts,
        "slice_end_ns": slice_end_ts,
        "session_start_ts": session_start_ts,
        "slice_before_data_start": slice_before_data_start,
        "effective_start_ns": effective_start_ts,
        "rows_in_slice": len(rows_in_slice),
        "unique_label_count": unique_label_count,
        "unique_labels": unique_labels,
        "unique_fixID" : unique_fixationIDs,

    }

    return result

In [66]:
def analyse_end_slice(
    session_name: str,
    slice_start_ts: int,
    slice_end_ts: int
):
    
    # ------------------------------------------------------------------
    # 1️⃣  Isolate the required session
    # ------------------------------------------------------------------
    label_col = "fixation_sum_label"
    sess_mask = allLabledData["session"].astype(str).str.startswith(session_name)
    sess_df = allLabledData.loc[sess_mask].copy()
    
    if sess_df.empty:
        raise ValueError(f"No rows found for session '{session_name}'")


    # ------------------------------------------------------------------
    # 2️⃣  First row of the session – this is the reference point
    # ------------------------------------------------------------------
    session_end_ts = sess_df.iloc[-1]["timestamp_[ns]"]

    # ------------------------------------------------------------------
    # 3️⃣  Does the slice start *before* the first data point?
    # ------------------------------------------------------------------
    slice_after_data_end = slice_end_ts > session_end_ts

    # ------------------------------------------------------------------
    # 4️⃣  Define the part of the slice we really can look at
    # ------------------------------------------------------------------
    # If the slice begins before the first fixation we can only start at the
    # first fixation timestamp.
    effective_end_ts = max(slice_end_ts, session_end_ts)

    # ------------------------------------------------------------------
    # 5️⃣  Select rows that lie inside the (possibly trimmed) slice
    # ------------------------------------------------------------------
    mask = (
        (sess_df["timestamp_[ns]"] > slice_start_ts) &
        (sess_df["timestamp_[ns]"] <= effective_end_ts)
    )
    rows_in_slice = sess_df.loc[mask]

    # ------------------------------------------------------------------
    # 6️⃣  Unique labels inside the slice
    # ------------------------------------------------------------------
    unique_labels = rows_in_slice[label_col].dropna().unique().tolist()
    unique_label_count = len(unique_labels)

    unique_fixationIDs = rows_in_slice["fixation_id"].dropna().unique().tolist()

    # ------------------------------------------------------------------
    # 7️⃣  Build the result dictionary
    # ------------------------------------------------------------------
    result = {
        "session": session_name,
        "slice_start_ns": slice_start_ts,
        "slice_end_ns": slice_end_ts,
        "session_end_ts": session_end_ts,
        "slice_after_data_end": slice_after_data_end,
        "effective_end_ns": effective_end_ts,
        "rows_in_slice": len(rows_in_slice),
        "unique_label_count": unique_label_count,
        "unique_labels": unique_labels,
        "unique_fixID" : unique_fixationIDs,
    }

    return result

In [67]:
overview = []          # list of dicts → will become a DataFrame later
problem_checkIndex = 0
allFilesCounter    = 0

sliceCheck_start = []
sliceCheck_end = []

# ------------------------------------------------------------------
# 4️⃣  Iterate over the sessions / files
# ------------------------------------------------------------------
for indexSess in range(1, 6):                     # sessions 1‑5
    # files that belong to the current session
    session_files = sorted(
        [
            f for f in os.listdir(datapath)
            if f.startswith(f"fixation_GPS_Expl_{indexSess}") and f.endswith("_labelled.csv")
        ]
    )

    for indexET, filename in enumerate(session_files, start=1):
        allFilesCounter += 1
        print(f"⏳  Session {indexSess} – ET {indexET} – {filename}")

        # --------------------------------------------------------------
        # 4.1 read the eye‑tracking data
        # --------------------------------------------------------------
        data_path = os.path.join(datapath, filename)

        data = pd.read_csv(data_path)


        # check start ts
        first_row = data.iloc[0]                       # pandas Series
        data_start_ts = first_row["timestamp_[ns]"]      # start of first sample
        data_end_ts   = first_row["end_timestamp_ns"]        # end   of first sample
        # identify correct slicing row
        sess_mask = slicingList["session"].astype(str).str.startswith(f"Expl_{indexSess}_ET_{indexET}_")
        slicingRow = slicingList.loc[sess_mask].copy()

        
        slice_start_ts = slicingRow["adjusted_start_timestamp_ns"].iloc[0]
        slice_end_ts = slicingRow["adjusted_end_timestamp_ns"].iloc[0]
        results_start = {}        
        results_end = {}  




        # --------------------------------------------------------------
        if slice_start_ts < data_start_ts:
            status_start = "slicing ts before data start"
            action_start = "check start slice"
            results_start = analyse_start_slice(f"Expl_{indexSess}_ET_{indexET}_", slice_start_ts, data_start_ts)
            sliceCheck_start.append(results_start)

            
        elif slice_start_ts > data_start_ts:
            if slice_start_ts >= data_end_ts:
                status_start = ("slicing ts after data start "
                      "but before first sample ends")
                action_start = "nothing"
            else:
                status = ("slicing ts after data start"
                      "and after first sample")
                secondRow = data.iloc[1]  
                
                if slice_start_ts > secondrow["end_timestamp_ns"]:
                    
                    action_start = "remove at least 2 start samples"
                    
                else:
                    action_start = "remove 1 start sample"
            

        else:
            status_start = ("same slicing TS")
            action_start = "nothing"


        # check ennd ts
        last_row = data.iloc[-1]                       # pandas Series
        secondLastRow = data.iloc[-2]          

        data_lastR_ts = last_row["timestamp_[ns]"]      # start of first sample
        data_lastR_end_ts   = last_row["end_timestamp_ns"]        # end   of first sample


        # --------------------------------------------------------------
        if slice_end_ts < data_lastR_ts:
            status_end = "end slice before last sample"
            if slice_end_ts >= secondLastRow["timestamp_[ns]"]:
                action_end = "remove 1 end sample"
                
            else:
                action_end = "remove at least 2 end samples"
                                
            
        elif slice_end_ts >= data_lastR_ts:
            if slice_end_ts <= data_lastR_end_ts:
                status_end = ("slicing ts after last sample "
                      "but before last sample ends")
                action_end = "nothing"
            else:
                status_end = ("slicing ts after last sample"
                      "and after last sample ends")
                action_end = "check end slice"
                results_end = analyse_end_slice(f"Expl_{indexSess}_ET_{indexET}_", data_lastR_end_ts, slice_end_ts)
                sliceCheck_end.append(results_end)

                
            

        else:
            status_end = ("same slicing TS")
            action_end = "nothing"

        # add info to overview
        overview.append(
            {
                "session":   indexSess,
                "et_index":  indexET,
                "file_name": filename,
                "data_start_ns":   data_start_ts,
                "data_end_ns":     data_end_ts,
                "slice_start_ns":  slice_start_ts,
                "status_start":  status_start,
                "status_end":  status_end,
                "action_start": action_start,
                "action_end": action_end
            }
        )

# ------------------------------------------------------------------
# 5️⃣  Convert the list of dicts to a DataFrame – the final overview
# ------------------------------------------------------------------
overview_df = pd.DataFrame(overview)
start_overview_df  = pd.DataFrame(sliceCheck_start)  # results of analyse_start_slice
end_overview_df    = pd.DataFrame(sliceCheck_end)    # results of analyse_end_slice



print("\n✅  Finished processing")
print(f"   – {allFilesCounter} files inspected")
print(f"   – {problem_checkIndex} files could not be read")

# Show the first rows for a quick sanity check

⏳  Session 1 – ET 1 – fixation_GPS_Expl_1_ET_1_labelled.csv
⏳  Session 1 – ET 2 – fixation_GPS_Expl_1_ET_2_labelled.csv
⏳  Session 1 – ET 3 – fixation_GPS_Expl_1_ET_3_labelled.csv
⏳  Session 2 – ET 1 – fixation_GPS_Expl_2_ET_1_labelled.csv
⏳  Session 2 – ET 2 – fixation_GPS_Expl_2_ET_2_labelled.csv
⏳  Session 2 – ET 3 – fixation_GPS_Expl_2_ET_3_labelled.csv
⏳  Session 3 – ET 1 – fixation_GPS_Expl_3_ET_1_labelled.csv
⏳  Session 3 – ET 2 – fixation_GPS_Expl_3_ET_2_labelled.csv
⏳  Session 3 – ET 3 – fixation_GPS_Expl_3_ET_3_labelled.csv
⏳  Session 4 – ET 1 – fixation_GPS_Expl_4_ET_1_labelled.csv
⏳  Session 4 – ET 2 – fixation_GPS_Expl_4_ET_2_labelled.csv
⏳  Session 5 – ET 1 – fixation_GPS_Expl_5_ET_1_labelled.csv
⏳  Session 5 – ET 2 – fixation_GPS_Expl_5_ET_2_labelled.csv

✅  Finished processing
   – 13 files inspected
   – 0 files could not be read


In [46]:
print(slicingRow["adjusted_start_timestamp_ns"])
print(slicingRow["adjusted_end_timestamp_ns"])

slicingRow.head()

12    1694101977583033333
Name: adjusted_start_timestamp_ns, dtype: int64
12    1694102631294966666
Name: adjusted_end_timestamp_ns, dtype: int64


,session,adjusted_start_frame,adjusted_start_timestamp,adjusted_start_timestamp_ns,adjusted_end_frame,adjusted_end_timestamp,adjusted_end_timestamp_ns
12,Expl_5_ET_2_2023-09-07_18-48-26_ET,8101,2023-09-07 15:52:57.583033333,1694101977583033333,27707,2023-09-07 16:03:51.294966666,1694102631294966666


In [68]:
overview_df.head(13)

,session,et_index,file_name,data_start_ns,data_end_ns,slice_start_ns,status_start,status_end,action_start,action_end
0,1,1,fixation_GPS_Expl_1_ET_1_labelled.csv,1693904646280860000,1693904646382090000,1693904632544755555,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
1,1,2,fixation_GPS_Expl_1_ET_2_labelled.csv,1693906797140610000,1693906797223920000,1693906793840633333,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
2,1,3,fixation_GPS_Expl_1_ET_3_labelled.csv,1693908900969360000,1693908901084320000,1693908870260911111,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
3,2,1,fixation_GPS_Expl_2_ET_1_labelled.csv,1693986185439750000,1693986185612540000,1693986184403811111,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
4,2,2,fixation_GPS_Expl_2_ET_2_labelled.csv,1693988007655300000,1693988007836090000,1693988004851122222,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
5,2,3,fixation_GPS_Expl_2_ET_3_labelled.csv,1693989995592270000,1693989995784460000,1693989967720244444,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
6,3,1,fixation_GPS_Expl_3_ET_1_labelled.csv,1693996182467380000,1693996182587620000,1693996179799455555,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
7,3,2,fixation_GPS_Expl_3_ET_2_labelled.csv,1693998168175670000,1693998168311960000,1693998166375244444,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
8,3,3,fixation_GPS_Expl_3_ET_3_labelled.csv,1693999977196220000,1693999977299270000,1693999974960144444,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice
9,4,1,fixation_GPS_Expl_4_ET_1_labelled.csv,1694014581793850000,1694014581872310000,1694014580693866666,slicing ts before data start,slicing ts after last sampleand after last sam...,check start slice,check end slice


In [69]:
start_overview_df.head(13)

,session,slice_start_ns,slice_end_ns,session_start_ts,slice_before_data_start,effective_start_ns,rows_in_slice,unique_label_count,unique_labels,unique_fixID
0,Expl_1_ET_1_,1693904632544755555,1693904646280860000,1693904632744766666,True,1693904632744766666,2119,3,"[street, rider, vegetation]","[964, 965, 966, 967, 968, 969, 970, 971, 972, ..."
1,Expl_1_ET_2_,1693906793840633333,1693906797140610000,1693906794272600000,True,1693906794272600000,444,3,"[person, street, vehicle]","[662, 663, 664, 665, 666, 667, 668, 669, 670, ..."
2,Expl_1_ET_3_,1693908870260911111,1693908900969360000,1693908870728877777,True,1693908870728877777,4869,6,"[{'vehicle': 37.5714674226421, 'building': 44....","[582, 583, 584, 585, 586, 587, 588, 589, 590, ..."
3,Expl_2_ET_1_,1693986184403811111,1693986185439750000,1693986184571788888,True,1693986184571788888,147,1,[street],"[768, 769, 770]"
4,Expl_2_ET_2_,1693988004851122222,1693988007655300000,1693988004987155555,True,1693988004987155555,430,1,[street],"[611, 612, 613, 614, 615, 616, 617, 618, 619, ..."
5,Expl_2_ET_3_,1693989967720244444,1693989995592270000,1693989967852311111,True,1693989967852311111,4671,7,"[street, vegetation, {'person': 41.11325291721...","[892, 893, 894, 895, 896, 897, 898, 899, 900, ..."
6,Expl_3_ET_1_,1693996179799455555,1693996182467380000,1693996180867344444,True,1693996180867344444,229,2,"[person, street]","[659, 660, 661, 662, 663, 664]"
7,Expl_3_ET_2_,1693998166375244444,1693998168175670000,1693998168175677777,True,1693998168175677777,0,0,[],[]
8,Expl_3_ET_3_,1693999974960144444,1693999977196220000,1693999975192477777,True,1693999975192477777,287,3,"[street, fence, {'fence': 48.40482978723403, '...","[533, 534, 535, 536, 537, 538, 539]"
9,Expl_4_ET_1_,1694014580693866666,1694014581793850000,1694014580757844444,True,1694014580757844444,139,2,"[{'building': 30.918697810783343, 'person': 52...","[586, 587, 588, 589]"


In [64]:
end_overview_df.head(13)

,session,slice_start_ns,slice_end_ns,session_end_ts,slice_after_data_end,effective_end_ns,rows_in_slice,unique_label_count,unique_labels
0,Expl_1_ET_1_,1693905263816420000,1693905274520766666,1693905274358333333,True,1693905274520766666,1719,7,"[street, person, {'vehicle': 59.71037776213546..."
1,Expl_1_ET_2_,1693907414930560000,1693907425712655555,1693907425245233333,True,1693907425712655555,1653,5,"[street, vegetation, vehicle, person, fence]"
2,Expl_1_ET_3_,1693909578009320000,1693909586792900000,1693909586593522222,True,1693909586792900000,1442,3,"[street, person, vehicle]"
3,Expl_2_ET_1_,1693986811768770000,1693986820979755555,1693986820547688888,True,1693986820979755555,1420,6,"[street, vehicle, fence, vegetation, {'person'..."
4,Expl_2_ET_2_,1693988650257970000,1693988650263077777,1693988649663066666,True,1693988650263077777,0,0,[]
5,Expl_2_ET_3_,1693990608885220000,1693990611060400000,1693990610860300000,True,1693990611060400000,367,1,[person]
6,Expl_3_ET_1_,1693996822318050000,1693996824075322222,1693996823675444444,True,1693996824075322222,199,3,"[street, {'building': 50.540851888417315, 'per..."
7,Expl_3_ET_2_,1693998799365010000,1693998802551144444,1693998801983333333,True,1693998802551144444,452,2,"[{'vehicle': 53.28223447957967, 'building': 46..."
8,Expl_3_ET_3_,1694000604733480000,1694000619104188888,1694000618772200000,True,1694000619104188888,2382,11,"[street, person, {'motorcycle': 33.50585750876..."
9,Expl_4_ET_1_,1694015219120520000,1694015221867988888,1694015221703877777,True,1694015221867988888,487,2,"[street, person]"


In [30]:

# overview = []          # list of dicts → will become a DataFrame later
# problem_checkIndex = 0
# allFilesCounter    = 0

# # ------------------------------------------------------------------
# # 4️⃣  Iterate over the sessions / files
# # ------------------------------------------------------------------
# for indexSess in range(1, 6):                     # sessions 1‑5
#     # files that belong to the current session
#     session_files = sorted(
#         [
#             f for f in os.listdir(datapath)
#             if f.startswith(f"fixation_GPS_Expl_{indexSess}") and f.endswith("_labelled.csv")
#         ]
#     )

#     for indexET, filename in enumerate(session_files, start=1):
#         allFilesCounter += 1
#         print(f"⏳  Session {indexSess} – ET {indexET} – {filename}")

#         data_path = os.path.join(datapath, filename)
#         data = pd.read_csv(data_path)
        

#         # check start ts
#         first_row      = data.iloc[0]                 # pandas Series
#         data_start_ts  = first_row["timestamp_[ns]"]   # start of first sample
#         data_end_ts    = first_row["end_timestamp_ns"]# end of first sample

#         slice_start_ts = slicingList.loc[indexET, "adjusted_start_timestamp_ns"]

       
#         if slice_start_ts < data_start_ts:
#             status_start = "slicing ts before data start"
#         elif slice_start_ts > data_start_ts:
#             if slice_start_ts >= data_end_ts:
#                 status_start = ("slicing ts after data start "
#                                "but before first sample ends")
#             else:
#                 status_start = ("slicing ts after data start"
#                                "and after first sample")
#         else:
#             status_start = "same slicing TS"


#          # check end ts

#         last_row       = data.iloc[-1]
#         data_lastR_ts  = last_row["timestamp_[ns]"]    # start of last sample
#         data_lastR_end = last_row["end_timestamp_ns"] # end of last sample
#         slice_end_ts   = slicingList.loc[indexET, "adjusted_end_timestamp_ns"]



#         if slice_end_ts > data_lastR_ts:
#             if slice_end_ts > second_last_row["timestamp_[ns]"] :
#                 status_end = ("slicing ts before last sample"
#                               "but after second last sample")

#             else:
#                  status_end = ("slicing ts before last sample"
#                               "and before second last sample")
                
#         elif slice_end_ts < data_lastR_ts:
#             if slice_end_ts >= data_lastR_end:
#                 status_end = ("slicing ts after last sample "
#                              "but before last sample ends")
#             else:
#                 status_end = ("slicing ts after last sample"
#                              "and after last sample ends")
#         else:
#             status_end = "same slicing TS"

#         # check differences in ts
#         diff_start_ms = (slice_start_ts - data_start_ts) /1e6
#         diff_end_ms   = (slice_end_ts   - data_lastR_ts) / 1e6

       
#         # save into overview list
#         overview.append(
#             {
#                 "session":          indexSess,
#                 "et_index":         indexET,
#                 "file_name":        filename,
#                 "data_start_ns":    data_start_ts,
#                 "data_end_ns":      data_end_ts,
#                 "data_lastR_ts":    data_lastR_ts,
#                 "data_lastR_end_ns":data_lastR_end,
#                 "slice_start_ns":   slice_start_ts,
#                 "slice_end_ns":     slice_end_ts,
#                 "status_start":     status_start,
#                 "status_end":       status_end,
#                 "diff_start_ms":    diff_start_ms,
#                 "diff_end_ms":      diff_end_ms,
#             }
#         )


# overview_df = pd.DataFrame(overview)



# print("\n✅  Finished processing")
# print(f"   – {allFilesCounter} files inspected")
# print(f"   – {problem_checkIndex} files could not be read")
# print(f"   – overview written to: {output_path}")


In [31]:
# overview_df.head(13)